In [34]:
from keras.layers import Input, Conv1D, Flatten, Dropout, MaxPooling1D, Dense, BatchNormalization, Activation, UpSampling1D, Concatenate
from keras.utils.np_utils import to_categorical   
from keras.models import Model
from keras import optimizers
import numpy as np
import h5py

In [3]:
def get_data(data_x, data_y, l1, l2):
    data_x_temp, data_y_temp = list(), list()
    for i in range(data_x.shape[0]):
        if data_y[i] == l1:
            data_y_temp.append(0)
            data_x_temp.append(data_x[i])
        elif data_y[i] == l2:
            data_y_temp.append(1)
            data_x_temp.append(data_x[i])
            
    return np.expand_dims(data_x_temp, axis=2), np.array(data_y_temp)

In [52]:
hf = h5py.File('preprocessed.h5', 'r')
X_train = np.array(hf.get('X_train'))
X_test = np.array(hf.get('X_test'))
y_train = np.array(hf.get('y_train'))
y_test = np.array(hf.get('y_test'))
hf.close()

In [5]:
models = []
for labels in range(0, 24, 2):
    print("")
    
    X_train_temp, y_train_temp = get_data(X_train, y_train, labels, labels+1)
    X_test_temp, y_test_temp = get_data(X_test, y_test, labels, labels+1)

    inputs = Input(shape=(X_train_temp.shape[-2], 1))

    x = Conv1D(16, 17, strides=1, padding='same')(inputs)
    x = BatchNormalization(axis = -1)(x)
    x = Activation('selu')(x)
    x = MaxPooling1D(2)(x)

    x = Conv1D(16, 11, strides=1, padding='same')(x)
    x = BatchNormalization(axis = -1)(x)
    x = Activation('selu')(x)
    x = MaxPooling1D(2)(x)

    x = Conv1D(16, 11, strides=1, padding='same')(x)
    x = BatchNormalization(axis = -1)(x)
    x = Activation('selu')(x)
    x = MaxPooling1D(2)(x)

    x = Conv1D(16, 7, strides=1, padding='same')(x)
    x = BatchNormalization(axis = -1)(x)
    x = Activation('selu')(x)
    x = MaxPooling1D(2)(x)

    x = Conv1D(16, 7, strides=1, padding='same')(x)
    x = BatchNormalization(axis = -1)(x)
    x = Activation('selu')(x)
    x = MaxPooling1D(2)(x)

    x = Flatten()(x)
    x = Dense(1, activation='linear')(x)
    x = Activation('sigmoid')(x)

    models.append(Model(inputs=inputs, outputs=x))
    models[-1].compile(loss='binary_crossentropy', optimizer=optimizers.Adam(lr=1e-2, decay=1e-5), metrics=['acc'])
    models[-1].fit(x=X_train_temp, y=y_train_temp, epochs=10, validation_data=(X_test_temp, y_test_temp), verbose=2)


Train on 65 samples, validate on 15 samples
Epoch 1/10
 - 2s - loss: 1.1611 - acc: 0.4769 - val_loss: 0.0133 - val_acc: 1.0000
Epoch 2/10
 - 0s - loss: 0.1336 - acc: 0.9692 - val_loss: 0.0200 - val_acc: 1.0000
Epoch 3/10
 - 0s - loss: 0.0248 - acc: 1.0000 - val_loss: 0.0029 - val_acc: 1.0000
Epoch 4/10
 - 0s - loss: 0.0125 - acc: 1.0000 - val_loss: 5.9657e-04 - val_acc: 1.0000
Epoch 5/10
 - 0s - loss: 0.0028 - acc: 1.0000 - val_loss: 4.3957e-04 - val_acc: 1.0000
Epoch 6/10
 - 0s - loss: 0.0018 - acc: 1.0000 - val_loss: 7.0395e-04 - val_acc: 1.0000
Epoch 7/10
 - 0s - loss: 0.0015 - acc: 1.0000 - val_loss: 6.2072e-04 - val_acc: 1.0000
Epoch 8/10
 - 0s - loss: 0.0014 - acc: 1.0000 - val_loss: 5.4681e-04 - val_acc: 1.0000
Epoch 9/10
 - 0s - loss: 0.0010 - acc: 1.0000 - val_loss: 2.9697e-04 - val_acc: 1.0000
Epoch 10/10
 - 0s - loss: 8.0102e-04 - acc: 1.0000 - val_loss: 2.0667e-04 - val_acc: 1.0000

Train on 72 samples, validate on 8 samples
Epoch 1/10
 - 3s - loss: 0.3736 - acc: 0.8056 - 

In [60]:
def predict(x, models):
    preds = []
    for i in range(len(models)):
        preds.append(models[i].predict(np.reshape(x, (1, 333, 1))))
        
    if (np.min(preds) < (1-np.min(preds))):
        return np.argmin(preds)*2
    else:
        return (np.argmax(preds)*2)+1

X_test_temp, y_test_temp = get_data(X_test, y_test, 4, 5)

y_pred = []
for i in range(X_test_temp.shape[0]):
    print(predict(X_test_temp[i], models), y_test_temp[i])

4 0
10 0
10 0
10 0
14 1
14 1
14 1
14 1
14 1
14 1
14 1
